In [1]:
from community import community_louvain
import logging
import os
import gc
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_probability as tfp
import warnings
logger = tf.get_logger()

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
logging.basicConfig(encoding='utf-8', level=logging.DEBUG,
                    force = True)

In [3]:
def graph1_adjacency_matrix():
    return np.array([
        [0, 1, 1, 0, 0, 0, 1, 0, ],
        [1, 0, 0, 1, 0, 0, 0, 1, ],
        [1, 0, 0, 1, 1, 0, 0, 0, ],
        [0, 1, 1, 0, 0, 1, 0, 0, ],
        [0, 0, 1, 0, 0, 1, 1, 0, ],
        [0, 0, 0, 1, 1, 0, 0, 1, ],
        [1, 0, 0, 0, 1, 0, 0, 1, ],
        [0, 1, 0, 0, 0, 1, 1, 0, ],
    ])


In [4]:
def build_test_graph1():
    adj_mat = graph1_adjacency_matrix()
    N = len(adj_mat)

    G = nx.Graph()
    for i in range(N):
        G.add_node(i)

    for i in range(N):
        for j in range(N):
            if adj_mat[i][j] == 1:
                G.add_edge(i, j)
                G.add_edge(j, i)
    return G

In [5]:
def calc_W(G, C):
    n = len(G.nodes())
    m = len(G.edges())
    m_c = len(C.edges())
    W = np.zeros((n,n+m+m_c),dtype= 'int8')
    for i in range(n):
        W[i][i] = 1
    j = n
    for edge in G.edges():
        x = int(edge[0])
        y = int(edge[1])
        W[x][j] = 1
        W[y][j] = 1
        j+=1
    for edge in C.edges():
        x = int(edge[0])
        y = int(edge[1])
        W[x][j] = 1
        W[y][j] = 1
        j+=1
    return W

def calc_b(G, C):
    n = len(G.nodes())
    m = len(G.edges())
    m_c = len(C.edges())
    b = np.array([-1/2 for i in range(n)])
    b_m = np.array([-1 for i in range(m+m_c)])
    return np.concatenate((b,b_m))

def calc_w(G, C):
    n = len(G.nodes())
    m = len(G.edges())
    m_c = len(C.edges())
    w = np.array([-1 for i in range(n)])
    w_n = np.array([n for i in range(m)])
    w_c = np.array([-1 for i in range(m_c)])
    return np.concatenate((w,w_n,w_c))

def build_omega(G):
    n = len(G.nodes())
    max_degree = max([G.degree(i) for i in range(n)])
#     logging.debug("Max degree = {}".format(max_degree))
    return np.array([1 - G.degree(i)/max_degree + random.random()/1000 for i in range(n)])  # Works bad for sall graphs
#     return np.array([random.random() for i in range(n)])

def build_network(G):
    logging.info("Build network on graph G:{}.".format(str(G)))
    C = nx.complement(G)
    W = calc_W(G, C)
    b = calc_b(G, C)
    w = calc_w(G, C)
    omega = build_omega(G)
#     logging.debug("Initial omega = {}".format(omega))
    return (W,b,w, omega, C)


In [6]:
def get_result_nodes(omega, alpha = 0.5):
    return set(np.argwhere(omega.numpy() > alpha).reshape(-1))

def not_connected_nodes_exist_in_G(G, result_nodes):
    for v in G.nodes():
        if v not in result_nodes:
            node_not_connected_to_G = True
            for edge in G.neighbors(v):
                if edge in result_nodes:
                    node_not_connected_to_G = False
                    break
            if node_not_connected_to_G:
                return True
    return False
            
def graph_has_no_edges(G, result_nodes):
    for v in result_nodes:
        for edge in G.neighbors(v):
            if edge in result_nodes:
                return False
    return True
            
def result_is_valid(G, omega):
    result_nodes = get_result_nodes(omega)
    return graph_has_no_edges(G, result_nodes) and not not_connected_nodes_exist_in_G(G, result_nodes)
    
            
def train_network(G, max_epochs = 100):
    (W,b,w, omega, C) = build_network(G)
    previous_omega = omega
    n = len(G.nodes())
    W_t = tf.constant(W.T, dtype = 'float32')
    b = tf.constant(b, dtype = 'float32')
    w_t = tf.constant(w.T, dtype = 'float32')
    omega = tf.Variable(omega,
                        trainable=True,
                        constraint = lambda x: tf.clip_by_value(omega,0,1),
                        dtype = 'float32')
    e_n = tf.constant(np.ones((n)),dtype = 'float32')

    def network():
#         h = tf.sigmoid(omega) # To make variable be from 0 to 1
        h = tf.math.multiply(e_n,omega)
        h = tf.linalg.matvec(W_t,h)
        h = tf.add(h,b)
        h = tf.nn.relu(h)
#         h = tf.sigmoid(h)
        h = tf.tensordot(w_t,h, 1)
        return h

    def loss():
        h = network()
        return (h-h_d)**2
    

    h_d = -n*n/2
    optimizer=tf.optimizers.Adam(learning_rate=0.1,)
    
    epoch = 0
    while not result_is_valid(G, omega) and epoch < max_epochs:
        logging.debug("Epoch = {}".format(epoch))
        optimizer.minimize(loss, var_list=[omega])
#         logging.debug("Omega = {}".format(omega))
        previous_error = loss()
#         logging.debug("Loss = {}".format(previous_error))
        epoch+=1
        if (previous_omega == omega.numpy()).all():
            logging.debug("Solver stuck")
            break
        previous_omega = omega.numpy()
    if result_is_valid(G, omega):
        logging.debug("Optimal solution found. Cardinality = {}".format(len(get_result_nodes(omega))))
    logging.debug("Total epochs = {}".format(epoch))
    return omega


In [7]:
def graphSets(graph): # Determinative algorithm to use correctness of the solution. Works too long for graphs >= 30 nodes.
      
    # Base Case - Given Graph 
    # has no nodes
    if(len(graph) == 0):
        return []
     
    # Base Case - Given Graph
    # has 1 node
    if(len(graph) == 1):
        return [list(graph.keys())[0]]
      
    # Select a vertex from the graph
    vCurrent = list(graph.keys())[0]
      
    # Case 1 - Proceed removing
    # the selected vertex
    # from the Maximal Set
    graph2 = dict(graph)
      
    # Delete current vertex 
    # from the Graph
    del graph2[vCurrent]
      
    # Recursive call - Gets 
    # Maximal Set,
    # assuming current Vertex 
    # not selected
    res1 = graphSets(graph2)
      
    # Case 2 - Proceed considering
    # the selected vertex as part
    # of the Maximal Set
  
    # Loop through its neighbours
    for v in graph[vCurrent]:
          
        # Delete neighbor from 
        # the current subgraph
        if(v in graph2):
            del graph2[v]
      
    # This result set contains VFirst,
    # and the result of recursive
    # call assuming neighbors of vFirst
    # are not selected
    res2 = [vCurrent] + graphSets(graph2)
      
    # Our final result is the one 
    # which is bigger, return it
    if(len(res1) > len(res2)):
        return res1
    return res2

def calculate_recursive_broot_force_result(G, draw_graph = True):
    dict_G = nx.to_dict_of_lists(G)
    print("Recursive approach")
    MIS = graphSets(dict_G)
    print(MIS)
    print("Length = "+str(len(MIS)))
    if draw_graph:
        color_map = ['red' if node in MIS else 'blue' for node in G]
        graph = nx.draw(G, node_color=color_map) # node lables
        plt.show()

In [8]:
def calculate_determenistic_result(G, draw_graph = True):
    print("Deterministic built-in approach")
    MIS = nx.maximal_independent_set(G)
    print(MIS)
    print("Length = "+str(len(MIS)))
    if draw_graph:
        color_map = ['yellow' if node in MIS else 'blue' for node in G]
        graph = nx.draw(G, node_color=color_map) # node lables
        plt.show()

In [9]:
def calculate_DNN_result(G, omega, threshold = 0.5, draw_graph = True):
    print("DNN approach")
    result_nodes = get_result_nodes(omega)
    print(result_nodes)
    print("Length = "+str(len(result_nodes)))
    if draw_graph:
        color_map = ['green' if node in result_nodes else 'blue' for node in G]        
        graph = nx.draw(G, node_color=color_map) # node lables
        plt.show()

In [10]:
def compareResults(G, max_epochs = 100, draw_graph = False):
#     calculate_recursive_broot_force_result(G)
    omega = train_network(G, max_epochs)
    calculate_determenistic_result(G, draw_graph)
    calculate_DNN_result(G, omega, draw_graph=draw_graph)

In [11]:
def read_G_from_file(filepath):
    return nx.read_edgelist(filepath)

In [12]:
# compareResults(build_test_graph1(), 100, True)

In [13]:
# compareResults(nx.complete_graph(10), draw_graph=False) # Expected result = 1

In [14]:
# compareResults(nx.graph_atlas(1000),draw_graph=True)

In [15]:
# compareResults(nx.gnp_random_graph(20, 0.1), 200, True) #Expected result around 12-14

In [16]:
# compareResults(nx.gnp_random_graph(50, 0.075), max_epochs = 1000)

In [17]:
# compareResults(nx.gnp_random_graph(100, 0.1), max_epochs = 1000, draw_graph=False)

In [18]:
wiki_vote_G = read_G_from_file('datasets/Wiki-Vote.txt') 

In [19]:
print(wiki_vote_G)

Graph with 7115 nodes and 100762 edges


In [20]:
def find_inter_cluster_edges(G, communities):
    edges = dict()
    for com in communities:
        for node_i in communities[com]:
            for neighbour_i in G.neighbors(node_i):
                if neighbour_i not in communities[com]:
                    if edges.get(node_i) is not None:
                        edges[node_i].append(neighbour_i)
                    else:
                        edges[node_i] = [neighbour_i]
                    if edges.get(neighbour_i) is not None:
                        edges[neighbour_i].append(node_i)
                    else:
                        edges[neighbour_i] = [node_i]
    return edges

def find_forbidden_edges(G, R, independent_sets):
    forbidden = []
    for node, edges in R.items():
        if node in independent_sets:
            for edge in edges:
                if edge in independent_sets:
                    forbidden.append((node, edge))
    return forbidden

def collect_list_by_dicts_key(partitions):
    communities = {}
    for key, val in partitions.items():
        if communities.get(val) == None:
            communities[val] = [key]
        else:
            communities[val].append(key)
    return communities

def build_G_from_nodes(G, nodes):
    communities = {}
    N = len(nodes)
    new_G = nx.Graph()
    index_map = dict()
    node_map = dict()
    index = 0
    for node in nodes:
        index_map[index] = node
        node_map[node] = index
        new_G.add_node(index)
        index+=1

    for i in nodes:
        for j in nodes:
            if G.has_edge(i,j):
                new_G.add_edge(node_map[i], node_map[j])
                new_G.add_edge(node_map[j], node_map[i])
    return (new_G, index_map, node_map)

In [21]:
def node_is_new_candidate(G, node, mis):
    neighbors = G.neighbors(node)
    neighbours_in_mis_count = 0
    for w in neighbors:
        if w in mis:
            neighbours_in_mis_count += 1
            if neighbours_in_mis_count == 2:
                break
            if neighbours_in_mis_count == 1:
                return (True, w)
    return (False, -1) 

def get_node_with_most_occurences(F, q, v):
    count_q = 0
    count_v = 0
    for x,y in F:
        if x == q:
            count_q+=1
        if y == v:
            count_v+=1
    if count_q > v:
        return q
    else:
        return v
            
def replace_node_if_possible(G,F,mis,node):
    if node not in mis:
        return True
    (can_be_replaced, new_node) = node_is_new_candidate(G,node,mis)
    if can_be_replaced:
        mis.remove(node)
        mis.add(new_node)
        return True
    return False 
        
def replace_forbiden_nodes(G,R,F,mis):
    while len(F) > 0:
        logging.debug("Length of F = {}".format(len(F)))
        for q,v in F:
            replaced = replace_node_if_possible(G,F,mis,q)
            if replaced:
                break
            else:
                replaced = replace_node_if_possible(G,F,mis,v)
            if replaced:
                break
            else:
                node_to_be_removed = get_node_with_most_occurences(F,q,v)
                mis.remove(node_to_be_removed)
        F = find_forbidden_edges(G, R, mis)
    return mis

def build_G_from_left_nodes(G, mis):
    mis_with_neighbours = set()
    for node in mis:
        mis_with_neighbours.add(node)
        for neighbour in G.neighbors(node):
            mis_with_neighbours.add(neighbour)
    nodes_left_to_process = set(G.nodes()).difference(mis_with_neighbours)
    return build_G_from_nodes(G,nodes_left_to_process)

def calculate_mis_with_left_nodes(G, mis_list, max_epochs):
    (left_G, left_index_map, left_node_map)  = build_G_from_left_nodes(G, mis_list)
    omega = train_network(left_G, max_epochs)
    left_mis = get_result_nodes(omega)
    left_mis_correct = [left_index_map[node] for node in left_mis]
    mis_final = new_mis.union(left_mis)
    return mis_final

In [22]:
def calculate_large_G(G, max_epochs = 20):
    gc.collect()
    partitions = community_louvain.best_partition(G, resolution= 0.8)
    communities = collect_list_by_dicts_key(partitions)
    logging.info("Total communities {}".format(len(communities)))
    R = find_inter_cluster_edges(G, communities)
    mis_list = set()
    community_index = 1
    for com in communities:
        logging.info("Community {} processing...".format(community_index))
        (small_G,index_map, node_map) = build_G_from_nodes(G, communities[com])
        omega_for_small_G = train_network(small_G, max_epochs)
        mis = get_result_nodes(omega_for_small_G)
        mis_correct = [index_map[node] for node in mis]
        mis_list = mis_list.union(mis_correct)
        community_index += 1
    F = find_forbidden_edges(G, R, mis_list)
    replace_forbiden_nodes(G,R,F,mis_list)
    return calculate_mis_with_left_nodes(G, mis_list, max_epochs)

In [ ]:
wiki_vote_G = calculate_large_G(wiki_vote_G)

INFO:root:Total communities 34
INFO:root:Community 1 processing...
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Total epochs = 20
INFO:root:Community 2 processing...
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:ro